# Validation Example

In [1]:
from datetime import datetime
from pydantic import BaseModel, PositiveInt

In [2]:
class User(BaseModel):
    id: int  
    name: str = 'John Doe'  
    signup_ts: datetime | None  
    tastes: dict[str, PositiveInt]  

In [3]:
external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',  
    'tastes': {
        'wine': 9,
        b'cheese': 7,  
        'cabbage': '1',  
    },
}

In [4]:
user = User(**external_data)

In [5]:
user.id

123

In [6]:
user.model_dump()

{'id': 123,
 'name': 'John Doe',
 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
 'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1}}

# Example of validation error

In [12]:
from pydantic import ValidationError


external_data = {'id': 'not an int', 'tastes': {}}
try:
    User(**external_data)  
except ValidationError as e:
    print(e.errors())

[{'type': 'int_parsing', 'loc': ('id',), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'not an int', 'url': 'https://errors.pydantic.dev/2.9/v/int_parsing'}, {'type': 'missing', 'loc': ('signup_ts',), 'msg': 'Field required', 'input': {'id': 'not an int', 'tastes': {}}, 'url': 'https://errors.pydantic.dev/2.9/v/missing'}]


In [13]:
from typing import Annotated, Literal
from annotated_types import Gt
from pydantic import BaseModel


class Fruit(BaseModel):
    name: str  
    color: Literal['red', 'green']  
    weight: Annotated[float, Gt(0)]  
    bazam: dict[str, list[tuple[int, bool, float]]]  


print(
    Fruit(
        name='Apple',
        color='red',
        weight=4.2,
        bazam={'foobar': [(1, True, 0.1)]},
    )
)

name='Apple' color='red' weight=4.2 bazam={'foobar': [(1, True, 0.1)]}


# Serialization

In [14]:
class Meeting(BaseModel):
    when: datetime
    where: bytes
    why: str = 'No idea'


m = Meeting(when='2020-01-01T12:00', where='home')

In [15]:
m.model_dump(exclude_unset=True)

{'when': datetime.datetime(2020, 1, 1, 12, 0), 'where': b'home'}

In [16]:
m.model_dump(exclude={'where'}, mode='json')

{'when': '2020-01-01T12:00:00', 'why': 'No idea'}

In [17]:
m.model_dump_json(exclude_defaults=True)

'{"when":"2020-01-01T12:00:00","where":"home"}'

# JSON Schema

In [18]:
from datetime import datetime

from pydantic import BaseModel


class Address(BaseModel):
    street: str
    city: str
    zipcode: str


class Meeting(BaseModel):
    when: datetime
    where: Address
    why: str = 'No idea'

{'$defs': {'Address': {'properties': {'street': {'title': 'Street', 'type': 'string'}, 'city': {'title': 'City', 'type': 'string'}, 'zipcode': {'title': 'Zipcode', 'type': 'string'}}, 'required': ['street', 'city', 'zipcode'], 'title': 'Address', 'type': 'object'}}, 'properties': {'when': {'format': 'date-time', 'title': 'When', 'type': 'string'}, 'where': {'$ref': '#/$defs/Address'}, 'why': {'default': 'No idea', 'title': 'Why', 'type': 'string'}}, 'required': ['when', 'where'], 'title': 'Meeting', 'type': 'object'}


In [19]:
Meeting.model_json_schema()

{'$defs': {'Address': {'properties': {'street': {'title': 'Street',
     'type': 'string'},
    'city': {'title': 'City', 'type': 'string'},
    'zipcode': {'title': 'Zipcode', 'type': 'string'}},
   'required': ['street', 'city', 'zipcode'],
   'title': 'Address',
   'type': 'object'}},
 'properties': {'when': {'format': 'date-time',
   'title': 'When',
   'type': 'string'},
  'where': {'$ref': '#/$defs/Address'},
  'why': {'default': 'No idea', 'title': 'Why', 'type': 'string'}},
 'required': ['when', 'where'],
 'title': 'Meeting',
 'type': 'object'}

# Strict Mode is usefull

In [20]:
from datetime import datetime

from pydantic import BaseModel, ValidationError


class Meeting(BaseModel):
    when: datetime
    where: bytes


m = Meeting.model_validate({'when': '2020-01-01T12:00', 'where': 'home'})
print(m)

try:
    m = Meeting.model_validate(
        {'when': '2020-01-01T12:00', 'where': 'home'}, strict=True
    )
except ValidationError as e:
    print(e)
    """
    2 validation errors for Meeting
    when
      Input should be a valid datetime [type=datetime_type, input_value='2020-01-01T12:00', input_type=str]
    where
      Input should be a valid bytes [type=bytes_type, input_value='home', input_type=str]
    """

m_json = Meeting.model_validate_json(
    '{"when": "2020-01-01T12:00", "where": "home"}'
)
print(m_json)

when=datetime.datetime(2020, 1, 1, 12, 0) where=b'home'
2 validation errors for Meeting
when
  Input should be a valid datetime [type=datetime_type, input_value='2020-01-01T12:00', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/datetime_type
where
  Input should be a valid bytes [type=bytes_type, input_value='home', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/bytes_type
when=datetime.datetime(2020, 1, 1, 12, 0) where=b'home'


# Dataclasses, TypedDicts, and more

In [23]:
from datetime import datetime

from typing_extensions import NotRequired, TypedDict

from pydantic import TypeAdapter


class Meeting(TypedDict):
    when: datetime
    where: bytes
    why: NotRequired[str]


meeting_adapter = TypeAdapter(Meeting)
m = meeting_adapter.validate_python(  
    {'when': '2020-01-01T12:00', 'where': 'home'}
)
print(m)
#> {'when': datetime.datetime(2020, 1, 1, 12, 0), 'where': b'home'}
meeting_adapter.dump_python(m, exclude={'where'})  

{'when': datetime.datetime(2020, 1, 1, 12, 0), 'where': b'home'}


{'when': datetime.datetime(2020, 1, 1, 12, 0)}

In [24]:
meeting_adapter.json_schema()

{'properties': {'when': {'format': 'date-time',
   'title': 'When',
   'type': 'string'},
  'where': {'format': 'binary', 'title': 'Where', 'type': 'string'},
  'why': {'title': 'Why', 'type': 'string'}},
 'required': ['when', 'where'],
 'title': 'Meeting',
 'type': 'object'}

# Customization

In [25]:
from datetime import datetime, timezone
from typing import Any

from pydantic_core.core_schema import ValidatorFunctionWrapHandler

from pydantic import BaseModel, field_validator


class Meeting(BaseModel):
    when: datetime

    @field_validator('when', mode='wrap')
    def when_now(
        cls, input_value: Any, handler: ValidatorFunctionWrapHandler
    ) -> datetime:
        if input_value == 'now':
            return datetime.now()
        when = handler(input_value)
        # in this specific application we know tz naive datetimes are in UTC
        if when.tzinfo is None:
            when = when.replace(tzinfo=timezone.utc)
        return when

In [26]:
Meeting(when='2020-01-01T12:00+01:00')

Meeting(when=datetime.datetime(2020, 1, 1, 12, 0, tzinfo=TzInfo(+01:00)))

In [27]:
Meeting(when='now')

Meeting(when=datetime.datetime(2024, 10, 7, 11, 32, 19, 713658))

In [28]:
Meeting(when='2020-01-01T12:00')

Meeting(when=datetime.datetime(2020, 1, 1, 12, 0, tzinfo=datetime.timezone.utc))